In [1]:
import sys
nb_dir = os.path.split(os.getcwd())[0]
if nb_dir not in sys.path:
    sys.path.append(nb_dir)

import numpy as np
import pandas as pd
# import modin.pandas as mpd
import spacy
from src.configs import ModelConfigs, Languages
from src.utils import wordifier, TextPreprocessor, encode

from textacy.preprocessing import make_pipeline, remove, replace, normalize
from tqdm import trange
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder
from sklearn.utils import resample
import multiprocessing as mp
# import dask.dataframe as dask_df
from stqdm import stqdm
stqdm.pandas()

from tqdm import trange

import os
# os.environ["MODIN_ENGINE"] = "ray"  # Modin will use Ray

import vaex
pd.set_option("display.max_colwidth", None)

In [2]:
df = pd.read_excel("../data/test_de.xlsx")
# mdf = mpd.read_csv("../data/test_en.csv")
language = "English"
nlp = spacy.load(Languages[language].value, exclude=["parser", "ner", "pos", "tok2vec"])

In [3]:
prep = TextPreprocessor(
    language="English", 
    cleaning_steps=list(TextPreprocessor._cleaning_options().keys()),
    lemmatizer_when=None,
)

In [4]:
df["p_text"] = prep.fit_transform(df["text"])

2021-05-10 14:30:04.984 WARNING root: 
  command:

    streamlit run /Users/49796/miniconda3/envs/py38/lib/python3.8/site-packages/ipykernel_launcher.py [ARGUMENTS]
100%|██████████| 6269/6269 [00:02<00:00, 2793.61it/s]


In [6]:
X, y, X_names, y_names = encode(df["p_text"], df["label"]).values()

In [11]:
clf = LogisticRegression(
    penalty="l1",
    C=0.05,#ModelConfigs.PENALTIES.value[np.random.randint(len(ModelConfigs.PENALTIES.value))],
    solver="saga",
    multi_class="auto",
    max_iter=500,
    class_weight="balanced",
)

In [12]:
%%time
clf.fit(X, y)

CPU times: user 1min 23s, sys: 138 ms, total: 1min 23s
Wall time: 1min 24s
/Users/49796/miniconda3/envs/py38/lib/python3.8/site-packages/sklearn/linear_model/_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "


LogisticRegression(C=0.05, class_weight='balanced', max_iter=500, penalty='l1',
                   solver='saga')

In [14]:
n_instances, n_features = X.shape
n_classes = len(y_names)

# NOTE: the * 10 / 10 trick is to have "nice" round-ups
sample_fraction = np.ceil((n_features / n_instances) * 10) / 10

sample_size = min(
    # this is the maximum supported
    ModelConfigs.MAX_SELECTION.value,
    # at minimum you want MIN_SELECTION but in general you want
    # n_instances * sample_fraction
    max(ModelConfigs.MIN_SELECTION.value, int(n_instances * sample_fraction)),
    # however if previous one is bigger the the available instances take
    # the number of available instances
    n_instances,
)

# TODO: might want to try out something to subsample features at each iteration

# initialize coefficient matrices
pos_scores = np.zeros((n_classes, n_features), dtype=int)
neg_scores = np.zeros((n_classes, n_features), dtype=int)

for _ in trange(ModelConfigs.NUM_ITERS.value):

    # run randomized regression
    clf = LogisticRegression(
        penalty="l1",
        C=ModelConfigs.PENALTIES.value[np.random.randint(len(ModelConfigs.PENALTIES.value))],
        solver="liblinear",
        multi_class="auto",
        max_iter=500,
        class_weight="balanced",
    )

    # sample indices to subsample matrix
    selection = resample(np.arange(n_instances), replace=True, stratify=y, n_samples=sample_size)

    # fit
    try:
        clf.fit(X[selection], y[selection])
    except ValueError:
        continue

    # record coefficients
    if n_classes == 2:
        pos_scores[1] = pos_scores[1] + (clf.coef_ > 0.0)
        neg_scores[1] = neg_scores[1] + (clf.coef_ < 0.0)
        pos_scores[0] = pos_scores[0] + (clf.coef_ < 0.0)
        neg_scores[0] = neg_scores[0] + (clf.coef_ > 0.0)
    else:
        pos_scores += clf.coef_ > 0
        neg_scores += clf.coef_ < 0

  6%|▌         | 28/500 [01:01<27:33,  3.50s/it]/Users/49796/miniconda3/envs/py38/lib/python3.8/site-packages/sklearn/svm/_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
 31%|███       | 156/500 [06:18<13:54,  2.43s/it]


KeyboardInterrupt: 

In [ ]:
# normalize
pos_scores = pos_scores / ModelConfigs.NUM_ITERS.value
neg_scores = neg_scores / ModelConfigs.NUM_ITERS.value

# get only active features
pos_positions = np.where(pos_scores >= ModelConfigs.SELECTION_THRESHOLD.value, pos_scores, 0)
neg_positions = np.where(neg_scores >= ModelConfigs.SELECTION_THRESHOLD.value, neg_scores, 0)

# prepare DataFrame
pos = [(X_names[i], pos_scores[c, i], y_names[c]) for c, i in zip(*pos_positions.nonzero())]
neg = [(X_names[i], neg_scores[c, i], y_names[c]) for c, i in zip(*neg_positions.nonzero())]

posdf = pd.DataFrame(pos, columns="word score label".split()).sort_values(["label", "score"], ascending=False)
negdf = pd.DataFrame(neg, columns="word score label".split()).sort_values(["label", "score"], ascending=False)